In [1]:
import pandas as pd
import numpy as np
import talib as ta
###读入每日指数数据
BM_price=pd.read_csv('F:\\Courses\\HKUST_Semester_Spring\\MAFS5210\\project2\\CSI300_daily.csv',sep=',')
BM_weight=pd.read_csv('F:\\Courses\\HKUST_Semester_Spring\\MAFS5210\\project2\\CSI300.csv',sep=',')

##读入指数成分股每日指标数据
stocks_index_d=pd.read_csv('F:\\Courses\\HKUST_Semester_Spring\\MAFS5210\\project2\\index_daily.csv',sep=',')
stocks_price_d=pd.read_csv('F:\\Courses\\HKUST_Semester_Spring\\MAFS5210\\project2\\stocks_daily.csv',sep=',')


In [2]:
stocks_price_d=stocks_price_d.iloc[::-1]
stocks_index_d=stocks_index_d.iloc[::-1]

##caculate alpha mometum factors
stocks_price_d['MOM']=ta.MOM(stocks_price_d['close'],10)
stocks_price_d['APO']=ta.APO(stocks_price_d['close'],10,20,0)
stocks_price_d['ROC']=ta.ROC(stocks_price_d['close'],10)
stocks_price_d['RSI']=ta.RSI(stocks_price_d['close'],10)
stocks_price_d['vol_volatility_1w']=ta.STDDEV(stocks_price_d['vol'],5)
stocks_price_d['vol_volatility_2w']=ta.STDDEV(stocks_price_d['vol'],10)
stocks_price_d['vol_volatility_1m']=ta.STDDEV(stocks_price_d['vol'],20)
##caculate alpha volume factors
stocks_price_d['OBV']=ta.OBV(stocks_price_d['close'],stocks_price_d['vol'])
##caculate alpha volatility factors
stocks_price_d['volatility_1w']=ta.STDDEV(stocks_price_d['pct_chg'],5)
stocks_price_d['volatility_2w']=ta.STDDEV(stocks_price_d['pct_chg'],10)
stocks_price_d['volatility_1m']=ta.STDDEV(stocks_price_d['pct_chg'],20)

stocks_price_d=stocks_price_d.iloc[::-1]
stocks_index_d=stocks_index_d.iloc[::-1]

In [3]:
###set the trade_date as datetime index
BM_price['trade_date']=pd.to_datetime(BM_price['trade_date'],format='%Y%m%d')
BM_price=BM_price.set_index('trade_date')

BM_weight['trade_date']=pd.to_datetime(BM_weight['trade_date'],format='%Y%m%d')
BM_weight=BM_weight.set_index('trade_date')
BM_weight=BM_weight.sort_index(ascending=False)

stocks_index_d['trade_date']=pd.to_datetime(stocks_index_d['trade_date'],format='%Y%m%d')
stocks_index_d=stocks_index_d.set_index('trade_date')

stocks_price_d['trade_date']=pd.to_datetime(stocks_price_d['trade_date'],format='%Y%m%d')
stocks_price_d=stocks_price_d.set_index('trade_date')


Since that the available datas in alpha factors has NaN in month 2014-01 and on 2019-12-31 the BM_weight begin at 2014-01-30,
we update all the available datas beginning at 2014-01-30 ending at 2014-12-30.

In [27]:

BM_price_real=BM_price[BM_price.index>=pd.datetime(2014,1,30)]
BM_weight_real=BM_weight[(BM_weight.index>=pd.datetime(2014,1,30))&
                         (BM_weight.index<=pd.datetime(2019,12,30))]
stocks_index_d_real=stocks_index_d[stocks_index_d.index>=pd.datetime(2014,1,30)]
stocks_price_d_real=stocks_price_d[stocks_price_d.index>=pd.datetime(2014,1,30)]

In [5]:
stocks_price_d_real.to_csv('F:\\Courses\\HKUST_Semester_Spring\\MAFS5210\\project2\\stocks_price_d_real.csv', 
                        sep=',', header=True, index=True)

In [6]:
stocks_index_d_real.to_csv('F:\\Courses\\HKUST_Semester_Spring\\MAFS5210\\project2\\stocks_index_d_real.csv', 
                        sep=',', header=True, index=True)

In [7]:
BM_price_real.to_csv('F:\\Courses\\HKUST_Semester_Spring\\MAFS5210\\project2\\BM_price_real.csv', 
                        sep=',', header=True, index=True)

In [28]:
BM_weight_real.to_csv('F:\\Courses\\HKUST_Semester_Spring\\MAFS5210\\project2\\BM_weight_real.csv', 
                        sep=',', header=True, index=True)

In [30]:
tradedate_d

DatetimeIndex(['2019-12-30', '2019-12-27', '2019-12-26', '2019-12-25',
               '2019-12-24', '2019-12-23', '2019-12-20', '2019-12-19',
               '2019-12-18', '2019-12-17',
               ...
               '2014-02-19', '2014-02-18', '2014-02-17', '2014-02-14',
               '2014-02-13', '2014-02-12', '2014-02-11', '2014-02-10',
               '2014-02-07', '2014-01-30'],
              dtype='datetime64[ns]', name='trade_date', length=1443, freq=None)

In [29]:
#define tradedate respectively per month and per day
tradedate_m=BM_weight_real.index
tradedate_d=BM_price_real.index
tradedate_m=tradedate_m.drop_duplicates()
tradedate_m

DatetimeIndex(['2019-12-02', '2019-11-29', '2019-11-01', '2019-10-31',
               '2019-10-08', '2019-09-30', '2019-09-02', '2019-08-30',
               '2019-08-01', '2019-07-31',
               ...
               '2014-07-31', '2014-07-01', '2014-06-30', '2014-05-30',
               '2014-05-05', '2014-04-30', '2014-04-01', '2014-03-31',
               '2014-02-28', '2014-01-30'],
              dtype='datetime64[ns]', name='trade_date', length=128, freq=None)

In [31]:
#Product a well data framework
Data=pd.DataFrame([])
total_m=len(tradedate_m)
for i in range(total_m):
        
    stocks_wgt_m=BM_weight_real[BM_weight_real.index==tradedate_m[i]][['con_code','weight']]
    tdm=tradedate_m[i]
    ts_code=stocks_wgt_m['con_code']
    new_index=pd.DataFrame([])
    new_price=pd.DataFrame([])
    print(tdm)
    print('Datas of the {} constituent stocks for current month is going to be arranged'.format(len(ts_code)))
    if i == 0:
        for j in range(len(ts_code)):
            ##get the daily index datas of stocks forming the benchmark
            new_index_temp=stocks_index_d_real[
                    (stocks_index_d_real.index>tradedate_m[i])&
                    (stocks_index_d_real.ts_code == ts_code.iloc[j])]
            new_index=pd.concat([new_index,new_index_temp],axis=0,sort=False)
            
            ##get the daily price datas of stocks forming the benchmark
            new_price_temp= stocks_price_d_real[
                    (stocks_price_d_real.index>tradedate_m[i])&
                    (stocks_price_d_real.ts_code == ts_code.iloc[j])]
            new_price=pd.concat([new_price,new_price_temp],axis=0,sort=False)
        ##add the weight of the stocks in the benchmark
        #stocks_wgt_m['weight']=stocks_wgt_m['weight']/sum(stocks_wgt_m['weight'])
        new_price['weight']= np.linspace(1,len(new_price.ts_code),len(new_price.ts_code))
        for k in range(len(ts_code)):
            new_price.loc[new_price['ts_code']==stocks_wgt_m['con_code'].iloc[k],'weight']=stocks_wgt_m['weight'].iloc[k]
        ##merge index and stocks_price by columns
        Data_new=pd.concat([new_price,new_index],axis=1,sort=False)
        ##drop the duplicated columns(['Unnamed: 0','ts_code']).
        Data_new=Data_new.T.drop_duplicates().T
        Data=pd.concat([Data,Data_new],axis=0,sort=False)
        print('Arrangement well done for current month, {} months remain.'.format(len(tradedate_m)-i-1))    
    else:
        for j in range(len(ts_code)):
            ##get the daily index datas of stocks forming the benchmark
            new_index_temp=stocks_index_d_real[
                    (tradedate_m[i]<=stocks_index_d_real.index)&
                    (stocks_index_d_real.index<tradedate_m[i-1])&
                    (stocks_index_d_real.ts_code == ts_code.iloc[j])]
            new_index=pd.concat([new_index,new_index_temp],axis=0,sort=False)
            
            ##get the daily price datas of stocks forming the benchmark
            new_price_temp= stocks_price_d_real[
                    (tradedate_m[i]<=stocks_price_d_real.index)&
                    (stocks_price_d_real.index<tradedate_m[i-1])&
                    (stocks_price_d_real.ts_code == ts_code.iloc[j])]
            new_price=pd.concat([new_price,new_price_temp],axis=0,sort=False)
        ##add the weight of the stocks in the benchmark
        #stocks_wgt_m['weight']=stocks_wgt_m['weight']/sum(stocks_wgt_m['weight'])
        new_price['weight']= np.linspace(1,len(new_price.ts_code),len(new_price.ts_code))
        for k in range(len(ts_code)):
            new_price.loc[new_price['ts_code']==stocks_wgt_m['con_code'].iloc[k],'weight']=stocks_wgt_m['weight'].iloc[k]
        ##merge index and stocks_price by columns
        Data_new=pd.concat([new_price,new_index],axis=1,sort=False)
        ##drop the duplicated columns(['Unnamed: 0','ts_code']).
        Data_new=Data_new.T.drop_duplicates().T
        Data=pd.concat([Data,Data_new],axis=0,sort=False)
        
        ##Compute the number of missed constituent stocks
        
        if (len(ts_code)!=len(Data_new.loc[tdm,'weight'])):
            print('There are {} constituent stocks datas missed'
                  .format(len(ts_code)-len(Data_new.loc[tdm,'weight'])))
        print('Arrangement well done for current month, {} months remain.'.format(len(tradedate_m)-i-1))
        
        

2019-12-02 00:00:00
Datas of the 300 constituent stocks for current month is going to be arranged
Arrangement well done for current month, 127 months remain.
2019-11-29 00:00:00
Datas of the 300 constituent stocks for current month is going to be arranged
There are 1 constituent stocks datas missed
Arrangement well done for current month, 126 months remain.
2019-11-01 00:00:00
Datas of the 300 constituent stocks for current month is going to be arranged
Arrangement well done for current month, 125 months remain.
2019-10-31 00:00:00
Datas of the 300 constituent stocks for current month is going to be arranged
Arrangement well done for current month, 124 months remain.
2019-10-08 00:00:00
Datas of the 300 constituent stocks for current month is going to be arranged
There are 2 constituent stocks datas missed
Arrangement well done for current month, 123 months remain.
2019-09-30 00:00:00
Datas of the 300 constituent stocks for current month is going to be arranged
Arrangement well done fo

There are 12 constituent stocks datas missed
Arrangement well done for current month, 84 months remain.
2018-01-31 00:00:00
Datas of the 300 constituent stocks for current month is going to be arranged
There are 12 constituent stocks datas missed
Arrangement well done for current month, 83 months remain.
2018-01-02 00:00:00
Datas of the 300 constituent stocks for current month is going to be arranged
There are 10 constituent stocks datas missed
Arrangement well done for current month, 82 months remain.
2017-12-29 00:00:00
Datas of the 300 constituent stocks for current month is going to be arranged
There are 10 constituent stocks datas missed
Arrangement well done for current month, 81 months remain.
2017-12-01 00:00:00
Datas of the 300 constituent stocks for current month is going to be arranged
There are 15 constituent stocks datas missed
Arrangement well done for current month, 80 months remain.
2017-11-30 00:00:00
Datas of the 300 constituent stocks for current month is going to be

There are 11 constituent stocks datas missed
Arrangement well done for current month, 43 months remain.
2016-05-03 00:00:00
Datas of the 300 constituent stocks for current month is going to be arranged
There are 17 constituent stocks datas missed
Arrangement well done for current month, 42 months remain.
2016-04-29 00:00:00
Datas of the 300 constituent stocks for current month is going to be arranged
There are 19 constituent stocks datas missed
Arrangement well done for current month, 41 months remain.
2016-04-01 00:00:00
Datas of the 300 constituent stocks for current month is going to be arranged
There are 17 constituent stocks datas missed
Arrangement well done for current month, 40 months remain.
2016-03-31 00:00:00
Datas of the 300 constituent stocks for current month is going to be arranged
There are 17 constituent stocks datas missed
Arrangement well done for current month, 39 months remain.
2016-03-01 00:00:00
Datas of the 300 constituent stocks for current month is going to be

There are 36 constituent stocks datas missed
Arrangement well done for current month, 2 months remain.
2014-02-28 00:00:00
Datas of the 300 constituent stocks for current month is going to be arranged
There are 36 constituent stocks datas missed
Arrangement well done for current month, 1 months remain.
2014-01-30 00:00:00
Datas of the 300 constituent stocks for current month is going to be arranged
There are 35 constituent stocks datas missed
Arrangement well done for current month, 0 months remain.


In [226]:
##Add benchmark pct_chg to our well data-framework(Data)
Data['benchmark']=np.zeros(len(Data.index))
for i in range(len(tradedate_d)):
    tdd=tradedate_d[i]
    Data.loc[Data.index==tdd,'benchmark']=BM_price_real.loc[tdd,'pct_chg']

In [43]:
import tushare as ts
ts.set_token('3381e3a464cd28c5f072dd48abcfb469cd753e88b1cc5ab1f703e5c7')
pro=ts.pro_api()

In [48]:
##Add the industry sector to our data

Data['code']=Data['ts_code'].apply(lambda x:x[:6]).tolist()
Data['sector']=np.zeros(len(Data['code']))
cc=list(Data['code'])
ds=ts.get_industry_classified()

for i in range(len(Data['code'])):
    if i%10000==0:
        print(i)

    sector=ds[ds.code.str.contains(cc[i])].c_name
    if len(sector)!=0:
        Data['sector'].iloc[i]=sector.iloc[0]
    else:
        Data['sector'].iloc[i]='NaN'  
    #df2.loc[df2['code']==,'sector']

0


C:\Users\51722\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000


In [50]:
Data['pct_chg']=Data['pct_chg']/100

In [228]:
Data['benchmark']=Data['benchmark']/100

In [167]:
#Get the macro factor data
df_shibor = pro.shibor(start_date='20140130', end_date='20191231')
df_libor = pro.libor(curr_type='USD', start_date='20140130', end_date='20191231')
df_shibor_lpr = pro.shibor_lpr(start_date='20140130', end_date='20191231')

In [168]:
df_shibor['date']=pd.to_datetime(df_shibor['date'],format='%Y%m%d')
df_shibor=df_shibor.set_index('date')
df_libor['date']=pd.to_datetime(df_libor['date'],format='%Y%m%d')
df_libor=df_libor.set_index('date')
df_shibor_lpr['date']=pd.to_datetime(df_shibor_lpr['date'],format='%Y%m%d')
df_shibor_lpr=df_shibor_lpr.set_index('date')

,date,1y
0,2019-12-31,4.15
1,2019-12-30,4.20
2,2019-12-27,4.20
3,2019-12-26,4.25
4,2019-12-25,4.31
5,2019-12-24,4.31
6,2019-12-23,4.31
7,2019-12-20,4.31
8,2019-12-19,4.31
9,2019-12-18,4.31


transform the daily data into monthly data

In [87]:
import datetime

def dateRange( beginDate, endDate):
    """
    :param beginDate: string, like "2019-11"
    :param endDate:  string, like "2020-03"
    :return: list
    """
    dates = []
    dt = datetime.datetime.strptime(beginDate, "%Y-%m")
    date = datetime.datetime.strptime(endDate, "%Y-%m")
    if dt > date:
        dt, date = date, dt
    while dt <= date:
        if datetime.datetime.strptime(datetime.datetime.strftime(dt, "%Y-%m-%d")[:-3], "%Y-%m") not in dates:
            dates.append(dt)
        dt = dt + datetime.timedelta(days=1)
        if len(dates) > 366:
            break
    dates = sorted(list(set(dates)))
    return dates


data_month=dateRange(beginDate='2014-02',endDate='2019-12')

In [88]:
for i in range(len(data_month)):
    data_month[i]=datetime.datetime.strftime(data_month[i],"%Y-%m")


In [99]:
#Get the monthly datetime for our dataframe.
date_m=[]
for j in range(len(data_month)):
    temp=[]
    for i in range(len(tradedate_d)):
        if datetime.datetime.strftime(tradedate_d[i],"%Y-%m")==data_month[j]:
            temp.append(tradedate_d[i])
    date_m.append(datetime.datetime.strftime(temp[0],"%Y-%m-%d"))

In [143]:
Data.iloc[Data.index<=date_m[0]]

,Unnamed: 0,ts_code,open,high,low,close,pre_close,change,pct_chg,vol,...,pb,ps,dv_ratio,float_share,free_share,total_mv,circ_mv,benchmark,code,sector
trade_date,,,,,,,,,,,,,,,,,,,,,
2014-02-28,1382,002081.SZ,18.41,18.67,18.2,18.64,18.41,0.23,0.0125,69041.3,...,3.9406,1.5707,0.71,109752,52514.6,2.18983e+06,2.04578e+06,2178.971,002081,建筑建材
2014-02-28,1426,600348.SH,5.96,5.97,5.75,5.89,5.96,-0.07,-0.0117,168513,...,1.0116,0.1981,4.86,240500,100196,1.41654e+06,1.41654e+06,2178.971,600348,煤炭行业
2014-02-28,1185,600153.SH,6.56,6.66,6.47,6.65,6.52,0.13,0.0199,179335,...,1.3565,0.1632,2.26,223775,119990,1.4881e+06,1.4881e+06,2178.971,600153,物资外贸
2014-02-28,1399,000961.SZ,6.56,6.77,6.51,6.74,6.56,0.18,0.0274,106073,...,1.1311,0.6039,1.48,116387,31562,787124,784445,2178.971,000961,建筑建材
2014-02-28,1367,000333.SZ,38.2,38.95,37.78,38.94,38.16,0.78,0.0204,79789.1,...,2.0551,0.64,NaN,68631.8,68631.8,6.56654e+06,2.67252e+06,2178.971,000333,家电行业
2014-02-28,1376,600516.SH,6.92,7.02,6.8,6.98,6.95,0.03,0.0043,69658.1,...,2.1723,3.0374,3.2,153489,74017.1,1.19997e+06,1.07136e+06,2178.971,600516,其它行业
2014-02-28,1426,600028.SH,5.3,5.47,5.26,5.37,5.41,-0.04,-0.0074,1.75352e+06,...,1.1246,0.2247,4.54,9.10519e+06,534236,6.25956e+07,4.88949e+07,2178.971,600028,石油行业
2014-02-28,1412,600068.SH,3.86,3.88,3.82,3.88,3.87,0.01,0.0026,102267,...,1.0181,0.2527,3.48,348746,185535,1.35313e+06,1.35313e+06,2178.971,600068,建筑建材
2014-02-28,1145,600157.SH,4.51,4.51,4.31,4.46,4.52,-0.06,-0.0133,278071,...,0.9071,1.0214,6.73,143395,104490,788332,639541,2178.971,600157,煤炭行业


In [146]:
#get our monthly dataframe
Data_month=pd.DataFrame([])
shares=Data['code'].drop_duplicates()
for i in range(len(date_m)):
    print(i)
    temp=Data.loc[Data.index==date_m[i],:]
    for j in range(len(shares)):
        if len(Data_month.index)>0:
            temp.loc[temp.code==shares[j],'pct_chg']=sum(Data.loc[(Data.index<=date_m[i])&(Data.index>date_m[i-1])&(Data.code==shares[j]),'pct_chg'])
        else:
            temp.loc[temp.code==shares[j],'pct_chg']=sum(Data.loc[(Data.index<=date_m[i])&(Data.index>tradedate_d[-1])&(Data.code==shares[j]),'pct_chg'])  
                                                        
    Data_month = pd.concat([temp,Data_month],axis=0)


0


C:\Users\51722\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70


In [232]:
#for i in range(len(Data_month.index)):
#    Data_month.loc[Data_month.index==Data_month.index[i],'benchmark']=Data.loc[Data.index==Data_month.index[i],'benchmark']

In [245]:
#Get the Dataframe we need (60 months data)
Data_need=pd.DataFrame([])
for i in range(61):
    tdd=date_m[i]
    temp=Data_month.loc[tdd,:]
    temp.loc[tdd,'shibor_1m']=df_shibor.loc[tdd,'1m']
    temp.loc[tdd,'shibor_6m']=df_shibor.loc[tdd,'6m']
    temp.loc[tdd,'libor_1m']=df_libor.loc[tdd,'1m']
    temp.loc[tdd,'libor_6m']=df_libor.loc[tdd,'6m']
    temp.loc[tdd,'shibor_lpr_1y']=df_shibor_lpr.loc[tdd,'1y']
    Data_need=pd.concat([temp,Data_need],axis=0)

C:\Users\51722\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\51722\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [ ]:
Data_need

In [234]:
#for i in range(len(Data_need.index)):
#    Data_need.loc[Data_need.index==Data_need.index[i],'benchmark']=Data.loc[Data.index==Data_need.index[i],'benchmark']

In [396]:
Data1=Data_need.loc[:,['ts_code','open','high','low','close','pct_chg','vol','benchmark',
                       'sector','weight',
                        'MOM','vol_volatility_1m','volatility_1m',
                       'pe','circ_mv','dv_ratio',
                      'shibor_1m', 'shibor_6m', 'libor_1m', 'libor_6m','shibor_lpr_1y']]

In [397]:
Data1['index']=Data1.index

In [398]:
Data1=Data1.reset_index(drop=True)

In [399]:
Data1=Data1.rename(columns={'ts_code':'symbol','sector':'Sector',})

In [400]:
Data1=Data1.sort_index(by='symbol')


F:\Codewares\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  """Entry point for launching an IPython kernel.


In [401]:
Data1['benchmark']

747      0.006737
4970     0.003653
15333    0.001303
5927    -0.000553
6512    -0.002021
12773    0.067147
11075   -0.009075
6193     0.003577
8834     0.004772
476      0.010462
4244     0.002975
1574     0.010364
7586     0.003735
6670     0.005612
8008    -0.007306
16670    0.000088
3207     0.000368
14620    0.019724
9550     0.033538
9090    -0.005342
8126    -0.001152
2137     0.000733
2735     0.021218
95      -0.002453
1917    -0.004951
10493   -0.023936
14570    0.022025
4457    -0.011755
13177   -0.005119
4722    -0.000749
           ...   
597      0.010462
14083   -0.013616
5340    -0.003184
2224     0.000733
2952     0.000368
3415     0.001142
2838     0.021218
9443     0.000803
7018     0.001912
6459    -0.002021
13526   -0.009044
203     -0.002453
11855    0.007579
16974   -0.002630
15946    0.012162
13990    0.001836
1483     0.010364
3935     0.004651
1970    -0.004951
17034    0.011542
7636     0.003735
6796     0.005612
12113    0.007256
945      0.011166
11322    0

In [402]:
list(Data1.loc[:,'Sector'].drop_duplicates())

['金融行业',
 '房地产',
 '综合行业',
 '电力行业',
 '机械行业',
 '有色金属',
 '农林牧渔',
 '电子信息',
 '酒店旅游',
 '家电行业',
 '传媒娱乐',
 '汽车制造',
 '发电设备',
 '水泥行业',
 '商业百货',
 '电子器件',
 '生物制药',
 '农药化肥',
 '酿酒行业',
 '环保行业',
 '化工行业',
 '钢铁行业',
 '化纤行业',
 '煤炭行业',
 '飞机制造',
 '建筑建材',
 '食品行业',
 '次新股',
 'NaN',
 '仪器仪表',
 '服装鞋类',
 '家具行业',
 '交通运输',
 '其它行业',
 '印刷包装',
 '船舶制造',
 '医疗器械',
 '石油行业',
 '物资外贸',
 '塑料制品',
 '玻璃行业',
 '公路桥梁',
 '电器行业',
 '开发区',
 '供水供气']

In [403]:
print(Data1.isnull().any())

symbol               False
open                 False
high                 False
low                  False
close                False
pct_chg              False
vol                  False
benchmark            False
Sector               False
weight               False
MOM                  False
vol_volatility_1m    False
volatility_1m        False
pe                    True
circ_mv              False
dv_ratio              True
shibor_1m            False
shibor_6m            False
libor_1m              True
libor_6m              True
shibor_lpr_1y        False
index                False
dtype: bool


In [404]:
#Data=Data.fillna(method='bfill')
#print(Data.isnull().any())
Data1=Data1.fillna(method='bfill')
Data1['1/pe']=1/Data1['pe']

In [405]:
## z-score normalization
alpha=['MOM','vol_volatility_1m','volatility_1m','1/pe','circ_mv','dv_ratio']
for i in range(len(alpha)):
    Data1[alpha[i]] = (Data1[alpha[i]]-Data1[alpha[i]].mean())/(Data1[alpha[i]].std()) 

In [406]:
Data1.columns

Index(['symbol', 'open', 'high', 'low', 'close', 'pct_chg', 'vol', 'benchmark',
       'Sector', 'weight', 'MOM', 'vol_volatility_1m', 'volatility_1m', 'pe',
       'circ_mv', 'dv_ratio', 'shibor_1m', 'shibor_6m', 'libor_1m', 'libor_6m',
       'shibor_lpr_1y', 'index', '1/pe'],
      dtype='object')

In [408]:
Data1['composite']=Data1['MOM']+Data1['vol_volatility_1m']+Data1['volatility_1m']+Data1['dv_ratio']+Data1['1/pe']+Data1['circ_mv']

In [410]:
ddd=Data1.loc[Data1.loc[:,'index']=='2019-01-31',['symbol','weight','pct_chg','composite']]

In [415]:
portfolio=ddd.sort_index(by='composite',ascending=False).iloc[0:20,:]

F:\Codewares\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  """Entry point for launching an IPython kernel.


In [418]:
portfolio['Wgt_Port']=portfolio['composite']/sum(portfolio['composite'])

In [449]:
portfolio['Wgt_BM']=portfolio['weight']
p=portfolio[['symbol','Wgt_Port']]

In [596]:
Score=portfolio[['symbol','composite']]

In [598]:
Score.columns=['TICKER','Score']

In [599]:
Score.to_csv('F:/Courses/HKUST_Semester_Spring/MAFS5210/project2/data_for_R/Score.csv',header=True)

In [556]:
port_ret=Data1[['index','symbol','weight','pct_chg','benchmark','composite']]

In [557]:
trad_mon=list(port_ret.loc[:,'index'].drop_duplicates().astype(str))

In [593]:
Risk_output_1=pd.DataFrame([])
for i in range(len(trad_mon)):
    print(i)
    temp=port_ret.loc[port_ret['index']==trad_mon[i],:]
    portf=temp.sort_index(by='composite',ascending=False)
    portf['Wgt_Port']=np.zeros(len(portf.index))
    temp_20=portf.iloc[0:20,]
    bm=portf['benchmark'].iloc[0]
    for j in range(20):
        temp_20['Wgt_Port'].iloc[j]=temp_20['composite'].iloc[j]/sum(temp_20['composite'].iloc[0:20])
    ret=sum(temp_20['Wgt_Port']*temp_20['pct_chg'])
    data1 = pd.DataFrame({'Date':[trad_mon[i]],  
                        'Top_20':[ret],
                      'CSI_300':[bm],})
    Risk_output_1=pd.concat([Risk_output_1,data1],axis=0)

0


F:\Codewares\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  """
C:\Users\51722\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
F:\Codewares\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


1


F:\Codewares\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  """
C:\Users\51722\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
F:\Codewares\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


2


...

F:\Codewares\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  """
C:\Users\51722\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
F:\Codewares\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


59


F:\Codewares\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  """
C:\Users\51722\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
F:\Codewares\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


60


F:\Codewares\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  """
C:\Users\51722\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
F:\Codewares\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [595]:
Risk_output_1.to_csv('F:/Courses/HKUST_Semester_Spring/MAFS5210/project2/data_for_R/Risk_output_1.csv',header=True)

In [437]:
dd2=BM_weight_real.loc[BM_weight_real.index=='2019-01-31',:]

In [445]:
dd2['Wgt_BM']=dd2['weight']/100
dd2['symbol']=dd2['con_code']

F:\Codewares\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
F:\Codewares\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [446]:
d_weight=dd2[['symbol','Wgt_BM']]

In [461]:
d_weight=d_weight.reset_index(drop=True)

In [458]:

d_weight['Wgt_Port']=np.zeros(len(d_weight))

F:\Codewares\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [464]:
ss=list(p['symbol'])
ss

['601398.SH',
 '600519.SH',
 '601288.SH',
 '600028.SH',
 '601988.SH',
 '600816.SH',
 '600516.SH',
 '600157.SH',
 '600104.SH',
 '601318.SH',
 '000625.SZ',
 '600036.SH',
 '002450.SZ',
 '601166.SH',
 '601857.SH',
 '002601.SZ',
 '601328.SH',
 '600518.SH',
 '600019.SH',
 '601088.SH']

In [476]:
ss=list(p['symbol'])
for i in range(len(ss)):
    d_weight.loc[d_weight['symbol']==ss[i],
                 'Wgt_Port']=p.loc[p['symbol']==ss[i],'Wgt_Port'].values

In [478]:
d_weight['Wgt_Act']=d_weight['Wgt_Port']-d_weight['Wgt_BM']

In [256]:
Data1.to_csv('F:/Courses/HKUST_Semester_Spring/MAFS5210/project2/data_for_R/csi_prices_and_industry.csv',header=True)

In [625]:
Data_need.columns

Index(['Unnamed: 0', 'ts_code', 'open', 'high', 'low', 'close', 'pre_close',
       'change', 'pct_chg', 'vol', 'amount', 'MOM', 'APO', 'ROC', 'RSI',
       'vol_volatility_1w', 'vol_volatility_2w', 'vol_volatility_1m', 'OBV',
       'volatility_1w', 'volatility_2w', 'volatility_1m', 'weight',
       'turnover_rate', 'volume_ratio', 'pe', 'pb', 'ps', 'dv_ratio',
       'float_share', 'free_share', 'total_mv', 'circ_mv', 'benchmark', 'code',
       'sector', 'shibor_1m', 'shibor_6m', 'libor_1m', 'libor_6m',
       'shibor_lpr_1y'],
      dtype='object')

In [293]:
Rtn_stk=Data_need.loc[:,['code','sector','pct_chg']]
#Rtn_stk['Date']=Rtn_stk.index
#Rtn_stk=Rtn_stk.reset_index(drop=True)

In [294]:
Rtn_stk=Rtn_stk.fillna(method='bfill')

In [296]:
Rtn_stk.columns=['Ticker','Sct','Return']

In [318]:
formater="{0:.06f}".format
Rtn_stk['Return']=Rtn_stk['Return'].apply(formater)

In [320]:
Rtn_stk.to_csv('F:/Courses/HKUST_Semester_Spring/MAFS5210/project2/data_for_R/Rtn_stk1.csv',
               header=True)

In [297]:
Rtn_stk['Ticker']=Rtn_stk['Ticker'].apply(lambda x:str(x)+'.SH')

In [303]:
Rtn_stk['Sct']=Rtn_stk['Sct'].astype(str)

trade_date
2019-02-28    300017.SH
2019-02-28    002736.SH
2019-02-28    002773.SH
2019-02-28    002797.SH
2019-02-28    002925.SH
2019-02-28    300003.SH
2019-02-28    300142.SH
2019-02-28    300015.SH
2019-02-28    300059.SH
2019-02-28    300024.SH
2019-02-28    300033.SH
2019-02-28    300070.SH
2019-02-28    300072.SH
2019-02-28    300122.SH
2019-02-28    300124.SH
2019-02-28    002673.SH
2019-02-28    300136.SH
2019-02-28    002714.SH
2019-02-28    002558.SH
2019-02-28    002625.SH
2019-02-28    002624.SH
2019-02-28    002450.SH
2019-02-28    300251.SH
2019-02-28    002422.SH
2019-02-28    002456.SH
2019-02-28    002460.SH
2019-02-28    002466.SH
2019-02-28    002468.SH
2019-02-28    002475.SH
2019-02-28    002493.SH
                ...    
2015-11-30    600277.SH
2015-11-30    600348.SH
2015-11-30    000970.SH
2015-11-30    601929.SH
2015-11-30    601168.SH
2015-11-30    002653.SH
2015-11-30    600497.SH
2015-11-30    600108.SH
2015-11-30    002344.SH
2015-11-30    000024.SH
2015-

In [346]:
stocks=Data_need.loc[:,'ts_code'].drop_duplicates()
stocks=list(stocks)

In [331]:
dd=list(Data_need.index.drop_duplicates().astype(str))

In [335]:
len(stocks)

476

In [427]:
df=pd.DataFrame([])
k=0
for i in range(len(stocks)):
    print(i)
    new= Rtn_stk.loc[Rtn_stk.Ticker==stocks[i],]
    if len(new)>=60:
        df=pd.concat([df,new],axis=0)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [429]:
df.to_csv('F:/Courses/HKUST_Semester_Spring/MAFS5210/project2/data_for_R/Rtn_stk.csv',
               header=True)

In [482]:
stocks11=list(df.loc[:,'Ticker'].drop_duplicates())

,Ticker,Sct,Return
trade_date,,,
2019-02-28,600036.SH,金融行业,0.086395
2019-01-31,600036.SH,金融行业,0.150656
2018-12-28,600036.SH,金融行业,-0.121925
2018-11-30,600036.SH,金融行业,-0.009397
2018-10-31,600036.SH,金融行业,-0.052581
2018-09-28,600036.SH,金融行业,0.085857
2018-08-31,600036.SH,金融行业,-0.000172
2018-07-31,600036.SH,金融行业,0.108427
2018-06-29,600036.SH,金融行业,-0.082200


In [483]:
Weight=pd.DataFrame([])
for i in range(len(stocks11)):
    temp=d_weight.loc[d_weight['symbol']==stocks11[i],:]
    Weight=pd.concat([Weight,temp],axis=0)

In [485]:
Weight=Weight.reset_index(drop=True)

In [486]:
Weight.to_csv('F:/Courses/HKUST_Semester_Spring/MAFS5210/project2/data_for_R/Weight.csv',header=True)

In [843]:
dddd=Weight
ST=list(Weight.symbol)

In [844]:
dddd['sector']=np.zeros(len(dddd['symbol']))
for i in range(len(ST)):
    dddd.loc[dddd.symbol==ST[i],'sector']=Data_need.loc[Data_need['ts_code']==ST[i],'sector'].iloc[0]

In [845]:
dddd.to_csv('F:/Courses/HKUST_Semester_Spring/MAFS5210/project2/data_for_R/stock_level.csv',header=True)

In [540]:
Rtn_stk2_m

,Ticker,Return
Date,,
2019-02-28,601398.SH,3.271624
2019-02-28,600519.SH,3.271624
2019-02-28,601288.SH,3.271624
2019-02-28,600028.SH,3.271624
2019-02-28,601988.SH,3.271624
2019-02-28,600816.SH,3.271624
2019-02-28,600516.SH,3.271624
2019-02-28,600157.SH,3.271624
2019-02-28,600104.SH,3.271624


In [498]:
sto=list(p['symbol'])
Rtn_stk2=pd.DataFrame([])
for i in range(len(sto)):
    print(i)
    temp=stocks_price_d_real.loc[stocks_price_d_real['ts_code']==sto[i],['ts_code','pct_chg']]
    Rtn_stk2=pd.concat([Rtn_stk2,temp])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [499]:
Rtn_stk2.columns=['Ticker','Return']
Rtn_stk2.index.name='Date'

In [534]:
Rtn_stk2['Return']=Rtn_stk2['Return']/100

In [849]:
tradedate_d[-1]

Timestamp('2014-01-30 00:00:00')

In [861]:
#get our monthly dataframe
Rtn_stk2_m=pd.DataFrame([])
ssst=list(Rtn_stk2['Ticker'].drop_duplicates().astype(str))

for j in range(len(ssst)):
    print(j)
    temp=Rtn_stk2.loc[Rtn_stk2.Ticker==ssst[j],:]
    temp['Return_m']=np.zeros(len(temp.index))
    for i in range(len(date_m)):
        if i >0:    
            temp.loc[date_m[i],'Return_m']=sum(temp.loc[(temp.index<=date_m[i])&(temp.index>date_m[i-1]),'Return'])
        else:
            temp.loc[date_m[i],'Return_m']=sum(Rtn_stk2.loc[(Rtn_stk2.index<=date_m[i])&(Rtn_stk2.index>tradedate_d[-1]),'Return'])  
        aaa=pd.DataFrame(temp.loc[date_m[i],['Ticker','Return_m']])
        Rtn_stk2_m = pd.concat([aaa,Rtn_stk2_m],axis=0)

0


F:\Codewares\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\51722\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [862]:
Rtn_stk2_m=Rtn_stk2_m.loc[Rtn_stk2_m.index<='2019-02-28',:]

In [863]:
Rtn_stk2_m

,Ticker,Return_m
Date,,
2019-02-28,601088.SH,0.032339
2019-01-31,601088.SH,0.100963
2018-12-28,601088.SH,-0.041515
2018-11-30,601088.SH,-0.063816
2018-10-31,601088.SH,-0.009977
2018-09-28,601088.SH,0.103078
2018-08-31,601088.SH,0.008453
2018-07-31,601088.SH,-0.028440
2018-06-29,601088.SH,-0.020800


In [864]:
Rtn_stk2_m.to_csv('F:/Courses/HKUST_Semester_Spring/MAFS5210/project2/data_for_R/Rtn_stk2.csv',header=True)

In [526]:
Sector=Data_need.loc[:,['ts_code','sector']].drop_duplicates()
Sector=Sector.reset_index(drop=True)

In [529]:
Sector.columns=['Ticker','Sct']

In [530]:
Sector.to_csv('F:/Courses/HKUST_Semester_Spring/MAFS5210/project2/data_for_R/Sector.csv',header=True)

In [733]:
MVO=pd.read_csv('F:/Courses/HKUST_Semester_Spring/MAFS5210/project2/data_for_R/MVO.csv')
MTE=pd.read_csv('F:/Courses/HKUST_Semester_Spring/MAFS5210/project2/data_for_R/MTE.csv')

In [734]:
MVO.columns=['Ticker','MVO']
MTE.columns=['Ticker','Min_TE']
MVO['Ticker']=MVO['Ticker'].apply(lambda x:x[:6]).tolist()
MTE['Ticker']=MTE['Ticker'].apply(lambda x:x[:6]).tolist()
p['symbol']=p['symbol'].apply(lambda x:x[:6]).tolist()

F:\Codewares\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [718]:
p

,symbol,Wgt_Port
387,601398,0.093790
361,600519,0.093435
379,601288,0.076000
577,600028,0.062184
419,601988,0.058776
348,600816,0.058740
360,600516,0.057335
315,600157,0.055892
321,600104,0.045328
380,601318,0.042658


In [626]:
Data_need.columns

Index(['Unnamed: 0', 'ts_code', 'open', 'high', 'low', 'close', 'pre_close',
       'change', 'pct_chg', 'vol', 'amount', 'MOM', 'APO', 'ROC', 'RSI',
       'vol_volatility_1w', 'vol_volatility_2w', 'vol_volatility_1m', 'OBV',
       'volatility_1w', 'volatility_2w', 'volatility_1m', 'weight',
       'turnover_rate', 'volume_ratio', 'pe', 'pb', 'ps', 'dv_ratio',
       'float_share', 'free_share', 'total_mv', 'circ_mv', 'benchmark', 'code',
       'sector', 'shibor_1m', 'shibor_6m', 'libor_1m', 'libor_6m',
       'shibor_lpr_1y'],
      dtype='object')

In [646]:
out_3=Data_need.loc[Data_need.index=='2019-01-31',:]
out_3['ts_code']=out_3['ts_code'].apply(lambda x:x[:6]).tolist()

F:\Codewares\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [664]:
stocks111=pd.DataFrame(stocks11)

In [665]:
stocks111[0]=stocks111[0].apply(lambda x:x[:6]).tolist()
stocks111=list(stocks111[0])

In [724]:
p

,symbol,Wgt_Port
387,601398,0.093790
361,600519,0.093435
379,601288,0.076000
577,600028,0.062184
419,601988,0.058776
348,600816,0.058740
360,600516,0.057335
315,600157,0.055892
321,600104,0.045328
380,601318,0.042658


In [720]:
p.to_csv('F:\\Courses\\HKUST_Semester_Spring\\MAFS5210\\project2\\data_for_R\\portfolio_weight.csv',
                     sep=',')

In [812]:
stocks_33=list(out_3['code'])

In [793]:
out_3=Data.loc[Data.index=='2019-01-31',['ts_code','code','sector','MOM','vol_volatility_1m',
                     'volatility_1m','pe','circ_mv','dv_ratio','weight'],]


In [817]:
out_3['Wgt_Port']=np.zeros(len(out_3.index))
out_3['Min_TE']=np.zeros(len(out_3.index))
out_3['MVO']=np.zeros(len(out_3.index))
stocks_33=list(out_3['code'])
for i in range(len(stocks_33)):
    temp1=MVO.loc[MVO.Ticker==stocks_33[i],'MVO']
    temp2=MTE.loc[MTE.Ticker==stocks_33[i],'Min_TE']
    temp3=p.loc[p.symbol==stocks_33[i],'Wgt_Port']
    if len(temp3)>0:
        out_3.loc[out_3.code==stocks_33[i],'Wgt_Port']=temp3.values
    if len(temp2)>0:
        out_3.loc[out_3.code==stocks_33[i],'Min_TE']=temp2.values
    if len(temp1)>0:
        out_3.loc[out_3.code==stocks_33[i],'MVO']=temp1.values

In [807]:
out_3['weight']=out_3['weight']/100

In [805]:
list_f=['MOM','vol_volatility_1m','volatility_1m','pe','circ_mv','dv_ratio']
stocks_l=list(out_3.code)
for factor in list_f:   
    a_1,a_2=out_3[factor].quantile([1/3,2/3]).values
    for i in range(len(out_3.code)):
        if out_3.loc[out_3.code==stocks_l[i],factor].values<a_1:
            out_3.loc[out_3.code==stocks_l[i],factor]=1
        elif (out_3.loc[out_3.code==stocks_l[i],factor].values>=a_1)&(out_3.loc[out_3.code==stocks_l[i],factor].values<a_2):
            out_3.loc[out_3.code==stocks_l[i],factor]=2
        else:
            out_3.loc[out_3.code==stocks_l[i],factor]=3

In [823]:
out_3.to_csv('F:\\Courses\\HKUST_Semester_Spring\\MAFS5210\\project2\\data_for_R\\riskoutput_3.csv',
                    sep=',')

In [752]:
Min_TE_1=MTE.loc[MTE.Ticker==stocks111[2],'Min_TE']
Min_TE_1.iloc[0]

0.0

In [827]:
aa=out_3['sector'].drop_duplicates()

In [828]:
aa.to_csv('F:\\Courses\\HKUST_Semester_Spring\\MAFS5210\\project2\\data_for_R\\sct.csv',
                     sep=',')

F:\Codewares\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [714]:
output_3.to_csv('F:\\Courses\\HKUST_Semester_Spring\\MAFS5210\\project2\\data_for_R\\output_3.csv',
                     sep=',')

In [ ]:
BM_price=pd.read_csv('F:\\Courses\\HKUST_Semester_Spring\\MAFS5210\\project2\\CSI300_daily.csv',
                     sep=',')

In [802]:
out_3=Data.loc[Data.index=='2019-01-31',['ts_code','code','sector','MOM','vol_volatility_1m',
                     'volatility_1m','pe','circ_mv','dv_ratio','weight'],]

In [789]:
stocks_3=list(out_3['ts_code'])

KeyError: 'ts_code'

In [726]:
stocks_3

['300017',
 '002736',
 '002773',
 '002797',
 '002925',
 '300003',
 '300142',
 '300015',
 '300059',
 '300024',
 '300033',
 '300070',
 '300072',
 '300122',
 '300124',
 '002673',
 '300136',
 '002714',
 '002558',
 '002625',
 '002624',
 '002450',
 '300251',
 '002422',
 '002456',
 '002460',
 '002466',
 '002468',
 '002475',
 '002493',
 '002508',
 '002555',
 '002572',
 '002594',
 '002601',
 '002602',
 '300144',
 '600157',
 '300296',
 '600036',
 '600048',
 '600050',
 '600061',
 '600066',
 '600068',
 '600085',
 '600089',
 '600100',
 '600104',
 '600109',
 '600111',
 '600115',
 '600118',
 '600153',
 '002411',
 '600038',
 '600031',
 '300408',
 '600030',
 '300433',
 '600000',
 '600004',
 '600009',
 '600010',
 '600011',
 '600015',
 '600016',
 '600018',
 '600019',
 '600023',
 '600025',
 '600027',
 '600028',
 '600029',
 '002415',
 '000002',
 '002352',
 '000553',
 '000625',
 '000627',
 '000630',
 '000651',
 '000671',
 '000703',
 '000709',
 '000725',
 '000728',
 '000768',
 '000776',
 '000783',
 '000786',